<a href="https://colab.research.google.com/github/argonne-lcf/ai-science-training-series/blob/main/04_intro_to_llms/IntroLLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parallel and distributed Deep Learning

## Author: Marieme Ngom, Argonne National Laboratory
(combining and adapting materials/discussion evolved over time by Huihuo Zheng, Bethany Lusch, Asad Khan, Prasanna Balaprakash, Taylor Childers, Corey Adams, Kyle Felker, Varuni Sastry, Sam Foreman, Archit Vasan, Carlo Graziani, Tanwi Mallick, and Venkat Vishwanath)
## Outline 
1. Day 1
    - Evolution of computig systems
    - Parallel computing
    - Introduction to Deep Learning
    - ***Parallel computing in AI***


2. ***Day 2***
    - ***Parallel computing in AI***
    - Brief Introduction to LLMs
    - Hands-on LLM training


# Parallel Computing in AI 
### Recap Single GPU
![x0](images/mermaid-figure-1.png)

Distributed training is the process of training models across multiple GPUs or other accelerators, with the goal of speeding up the training process and enabling the training of larger models on larger datasets.

There are two ways of parallelization in distributed training. 
* ***Data parallelism***: 
    * Each worker (GPU) has a complete set of model
    * different workers work on different subsets of data. 
* *Model parallelism* 
    * The model is split into different parts and stored on different workers
    * Different workers work on computation involved in different parts of the model
![PI](images/parallel_computing.png)

## Scaling goal: 
1. Minimize cost i.e. amount of time spent training
2. Maximize performance i.e model quality metrics, throughput/efficiency metrics (images/seconds, GPU/CPU utilization percentages, flops efficiency)

## Training on multiple GPUs: Data Parallelism
### Nomenclature: 
- N = number of GPUs = WORLD_SIZE
- Each GPU is assigned a rank from 0 to WORLD_SIZE-1
- A worker = a GPU here
![mgpus](images/mermaid-figure-15.png)
*Each GPU receives unique data at each step*
### Data Parallel: Forward Pass
![forward](images/mermaid-figure-14.png)
*Average gradients across all GPUs*
### Data Parallel: Backward Pass
![backward](images/mermaid-figure-13.png)
*Send global updates back to each GPU*
### Data Parallel: Full Setup
![full](images/mermaid-figure-12.png)
*See: [PyTorch / Distributed Data Parallel](https://docs.pytorch.org/tutorials/intermediate/ddp_tutorial.html)*
### Data Parallel: Training
- Each GPU:
    - has identical copy of model
    - works on a unique subset of data
- Easy to get started with
    - [saforeman2/ezpz](https://github.com/saforem2/ezpz)
    - [PyTorch/DDP](https://docs.pytorch.org/docs/stable/notes/ddp.html)
    - [HF/Accelerate](https://huggingface.co/docs/transformers/accelerate)
    - [Microsoft/DeepSpeed](https://www.deepspeed.ai)
- Requires ***global*** communication
    - every rank must participate (collective communication)

## Communication
- Need mechanism(s) for communicating across GPUs:
    - [mpi4py](https://mpi4py.readthedocs.io/en/stable/tutorial.html)
    - [torch.distributed](https://docs.pytorch.org/docs/stable/distributed.html)
- Collective communication:
    - [Nvidia Collective Communications Library (NCCL)](https://developer.nvidia.com/nccl)
    - [Intel oneAPI Collective Communications](https://www.intel.com/content/www/us/en/developer/tools/oneapi/oneccl.html#gs.n9y302)
***Timeouts*** Collective operations have to be called for each rank to form a complete collective operation.
Failure to do so will result in other ranks waiting indefinitely
### AllReduce
![allreduce](images/mermaid-figure-11.png)

### Broadcast
![broadcast](images/mermaid-figure-9.png)

## Dealing with Data
- At each training step, we want to ensure that each worker receives unique data
- This can be done in one of two ways:
    1. Manually partition data (ahead of time)
        - Assign unique subsets to each worker
        - Each worker can only see their local portion of the data
        - Most common approach
    2. From each worker, randomly select a mini-batch
        - Each worker can see the full dataset
        - ⚠️ When randomly selecting, it is important that each worker uses different seeds to ensure they receive unique data

## Broadcast Initial State
- At the start of training (or when loading from a checkpoint), we want all of our workers to be initialized consistently
    - Broadcast the model and optimizer states from rank() == 0 worker
![bcast](images/mermaid-figure-6.png)

## Why distributed training?
- N workers each processing unique batch (micro batch size) of data:
    - (micro_batch_size = 1)× $N_{GPUs}$ → ***global_batch_size = N***
- Improved gradient estimators
    - Smooth loss landscape
    - Less iterations needed for same number of epochs
        - common to scale learning rate lr *= sqrt(N)
        
![speedup](images/speedup.png)

## Going Beyond Data Parallelism
- Useful when model fits on single GPU:
    - ultimately limited by GPU memory
    - model performance limited by size
- ⚠️ When model does not fit on a single GPU:
    - Offloading (can only get you so far…):
        - DeepSpeed + ZeRO
        - PyTorch + FSDP
- Otherwise, resort to [model parallelism strategies](https://samforeman.me/talks/ai-for-science-2024/slides#/additional-parallelism-strategies)


# Going beyond Data Parallelism:  DeepSpeed + ZeRO
- Depending on the ZeRO stage (1, 2, 3), we can offload
    1. ***Stage 1***: optimizer states (P_{os})
    2. ***Stage 2***: optimizer states+gradients (P_{os+g})
    2. ***Stage 3***: optimizer states+gradients+model params (P_{os+g+p})

![zero](images/zero.png)

# Model parallel training: example
Want to compute $y = \sum_i x_iW_i = x_0W_0 + x_1W_1 + x_2W_2$ where each GPU only has only its portion of the full weights as shown below
1. Compute $y_0=x0W_0$ -> **GPU1**
2. Compute $y_1=y_0 +x_1W_1$ -> **GPU2**
3. Compute $y_2=y_1 + x_2W_2$

![modelpar](images/mermaid-figure-2.png)

# Deciding on a parallelism strategy
![onedec](images/onegpudec.png)
![multgpu](images/onenodemulgpu.png)

![AIcompute](images/ai-and-compute-all-2.png.webp)

Sophia: 192 GPUs (8/node), 3.9 Petaflops ($10^15$)/s
![sophia](images/sophia.jpeg)
Polaris: 2240 GPUs (4/node), 78 Teraflops ($10^12$)/s
![polaris](images/polaris.jpeg)
Aurora: 63,744 GPUs (6/node), exascale computer ($10^18$ calculations per second)
![aurora](images/aurora.jpeg)

# Brief introduction to LLMs

## Training LLMs

## Life-cycle of a LLM
1. Data collection + preprocessing
2. ***Pre-training***
    - Architecture decisions, model size, etc.
3. Supervised Fine-Tuning
    - Instruction Tuning
    - Alignment
4. Deploy (+ monitor, re-evaluate, etc.)

![gptcycle](images/gpt3-training-step-back-prop.gif)
*Source:Figure from [The Illustrated Transformer](https://jalammar.github.io/illustrated-transformer/)*

## Life-cycle of a LLM
1. Data collection + preprocessing
2. Pre-training
    - Architecture decisions, model size, etc.
3. ***Supervised Fine-Tuning***
    - Instruction Tuning
    - Alignment
4. Deploy (+ monitor, re-evaluate, etc.)

![gptcycle2](images/gpt3-fine-tuning.gif)
*Source:Figure from [The Illustrated Transformer](https://jalammar.github.io/illustrated-transformer/)*

## Forward pass
![fwdpass](images/hf_assisted_generation.mov)
*Source: [Generation with LLMs](https://huggingface.co/docs/transformers/main/en/llm_tutorial)*

## Generating text
![fwdpass](images/hf_assisted_generation2.mov)
*Source: [Generation with LLMs](https://huggingface.co/docs/transformers/main/en/llm_tutorial)*

# Hands-on LLM Training


***Good practice*** (not needed here): Create and activate a conda (or virtual) environment 
```conda create -n env_mlss_dnn python=3.9```
then on jupyter do new ->terminal

```
 conda activate env_mlss_dnn
 pip install ipykernel 
 python -m ipykernel install --user --name env_mlss_dnn
```

then go back to your .ipynb file, change kernel to env_mlss_dnn.



In [ ]:
!git clone https://github.com/karpathy/nanoGPT.git

In [ ]:
%pwd

%cd nanoGPT

%pwd

In [ ]:
#!pip install torch numpy transformers datasets tiktoken wandb tqdm

| Dataset               | Tokens (≈)         | Disk size / notes                                 |
| --------------------- | ------------------ | ------------------------------------------------- |
| **openwebtext**       | 9 B tokens total   | 9 B train (≈17GB) / 4M val (≈8.5MB)                |
| **shakespeare (tiny)** | ≈ 330K tokens total | 301,966 train / 36,059 val                   |
| **shakespeare\_char** | 1,115,394 chars    | 1,003,854 train / 111,540 val (character‐level)   |


| model | params | train loss | val loss |
| ------| ------ | ---------- | -------- |
| gpt2 | 124M         | 3.11  | 3.12     |
| gpt2-medium | 350M  | 2.85  | 2.84     |
| gpt2-large | 774M   | 2.66  | 2.67     |
| gpt2-xl | 1558M     | 2.56  | 2.54     |


In [3]:
!python3 data/shakespeare_char/prepare.py

/Library/Frameworks/Python.framework/Versions/3.7/Resources/Python.app/Contents/MacOS/Python: can't open file 'data/shakespeare_char/prepare.py': [Errno 2] No such file or directory


In [ ]:
!python3 train.py config/train_shakespeare_char.py --compile=False --eval_iters=20 --log_interval=1 --block_size=64 --batch_size=16
--n_layer=4 --n_head=4 --n_embd=128 --max_iters=2000 --lr_decay_iters=2000 --dropout=0.0

In [ ]:
!pip install git+https://github.com/openai/whisper.git

In [ ]:
import tiktoken

In [ ]:
!python3 sample.py --out_dir=out-shakespeare-char

In [ ]:
!python3 train.py config/train_shakespeare_char.py #training longer

In [ ]:
!python3 sample.py --out_dir=out-shakespeare-char

In [ ]:
!export NCCL_DEBUG=INFO
!export NCCL_DEBUG_SUBSYS=ALL
!export NCCL_DEBUG_FILE=nccl_trace.log

# Running on NVIDIA T4 Tensor Cores, 4GPUS/node

In [ ]:
import socket
ip=socket.gethostbyname(socket.gethostname())
print(ip)

In [ ]:
!export CUDA_VISIBLE_DEVICES=0 #,1,2,3          
!export MASTER_ADDR=ip           
!export MASTER_PORT=29500                     

!torchrun \
  --nnodes=1 \
  --node_rank=0 \
  --nproc_per_node=12 \
  --master_addr=$ip \
  --master_port=29500 \
  train.py \
    config/train_shakespeare_char.py \
    --batch_size=64 \
    --gradient_accumulation_steps=40

![llms](images/llms.gif)
*Source: [Hannibal046/Awesome-LLM](https://github.com/Hannibal046/Awesome-LLM)*

![emergent](images/emergent-abilities.gif)


![evolllms](images/evolution.gif)